In [ ]:
!pip install praw unidecode pandas openpyxl

In [ ]:
import praw
import pandas as pd
from datetime import datetime
import time

# 🔹 Load Product Names from Excel
file_path = "/content/Headphones.xlsx"
sheet_name = "Headphone-Avail-Price"

# Read product names from the specified sheet
df_products = pd.read_excel(file_path, sheet_name=sheet_name)

# Ensure "Product Name" column exists
if "Product Name" not in df_products.columns:
    raise ValueError("❌ 'Product Name' column not found in the Excel sheet.")

# Extract product names
products = df_products["Product Name"].dropna().unique().tolist()

# 🔹 Reddit API Credentials (Ensure these are correct and not shared publicly)
reddit = praw.Reddit(
    client_id='jNAMMjqIULpTFdVMVeVxkg',
    client_secret='S5lIYbBLN0WDcoKvoYUgKzF3p2gB7g',
    user_agent='MyShopt by /u/TemporaryTop5884'
)

def get_all_comments_from_top_posts(search_query, min_comments=100, comments_per_post=None, wait_time=3):

    all_comments = []
    total_comments = 0

    while total_comments < min_comments:
        # 🔹 Fetch top Reddit posts for the query
        submissions = list(reddit.subreddit("all").search(
            search_query,
            sort="relevance",
            syntax="lucene",
            time_filter="all",
            limit=100  # Fetch up to 100 posts per query
        ))

        if not submissions:
            print(f"❌ No results found for '{search_query}'. Skipping...")
            break

        # 🔹 Process each submission
        for submission in submissions:
            print(f"📌 Processing post: {submission.title}")

            # Load full submission with comments
            full_submission = reddit.submission(id=submission.id)
            full_submission.comments.replace_more(limit=5)  # Expand comments
            flat_comments = full_submission.comments.list()

            # 🔹 Apply per-post comment limit (if set)
            if comments_per_post:
                flat_comments = flat_comments[:comments_per_post]

            # 🔹 Store comments
            for comment in flat_comments:
                all_comments.append({
                    "product": search_query,
                    "post_title": submission.title,
                    "post_id": submission.id,
                    "post_url": f"https://www.reddit.com{submission.permalink}",
                    "post_score": submission.score,
                    "post_created_utc": datetime.fromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    "subreddit": str(submission.subreddit),

                    "comment_id": comment.id,
                    "comment_author": str(comment.author),
                    "comment_body": comment.body,
                    "comment_score": comment.score,
                    "comment_created_utc": datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    "comment_url": f"https://www.reddit.com{comment.permalink}",
                    "comment_depth": comment.depth,
                    "is_top_level": comment.depth == 0
                })
                total_comments += 1

                # Stop if we hit the minimum required comments
                if total_comments >= min_comments:
                    break

            if total_comments >= min_comments:
                break

        # Avoid rate limits
        time.sleep(wait_time)

    return pd.DataFrame(all_comments)

# 🔹 Fetch & Save Data for Each Product
all_products_comments = []

# 🔹 Fetch Data for Each Product
for product in products:
    search_query = f"Review {product} headphone"
    print(f"\n🚀 Searching for: {search_query} ...")
    comments_df = get_all_comments_from_top_posts(search_query)

    if not comments_df.empty:
        all_products_comments.append(comments_df)
        print(f"✅ Found {len(comments_df)} comments for {product}")
    else:
        print(f"⚠️ No comments found for {product}.")

# 🔹 Combine all product comments into a single DataFrame
if all_products_comments:
    combined_df = pd.concat(all_products_comments, ignore_index=True)

    # Save as a single Excel file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"All_Headphone_Reviews_Reddit_Comments_{timestamp}.xlsx"
    combined_df.to_excel(filename, index=False)
    print(f"\n✅ Saved all {len(combined_df)} comments to: {filename}")

    # Optionally save each product as a separate sheet in the same Excel file
    with pd.ExcelWriter(f"Headphone_Reviews_By_Product_{timestamp}.xlsx") as writer:
        for product in products:
            product_df = combined_df[combined_df['product'] == f"Review {product} headphone"]
            if not product_df.empty:
                # Create sheet name (Excel has 31 character limit for sheet names)
                sheet_name = product[:31]
                product_df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"✅ Also saved comments by product in separate sheets")
else:
    print("\n⚠️ No comments found for any products.")

print("\n🎉 Done! All comments saved.")


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




🚀 Searching for: Review Sennheiser HD 800 S headphone ...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



📌 Processing post: HD 800 S (Good deal and upgrade from HD 600?)


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



...

📌 Processing post: OKCSC M2 Review
📌 Processing post: Anyone tried the wireless Grados?


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



📌 Processing post: What open back headphones should I buy
📌 Processing post: Help finding great bluetooth headphones for someone who doesn’t take care of things


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



📌 Processing post: I had the opportunity to test every Sennheiser headphone today! At the Sennheiser experience Brasil here is my review:
✅ Found 100 comments for Grado GW100x

✅ Saved all 14500 comments to: All_Headphone_Reviews_Reddit_Comments_20250404_231032.xlsx
✅ Also saved comments by product in separate sheets

🎉 Done! All comments saved.


In [ ]:
!pip uninstall numpy gensim -y


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3


In [ ]:
!pip install --upgrade numpy

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.


In [ ]:
!pip install --upgrade gensim



  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4


In [ ]:
!pip show gensim

Name: gensim
Version: 4.3.3
Summary: Python framework for fast Vector Space Modelling
Home-page: https://radimrehurek.com/gensim/
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: /usr/local/lib/python3.11/dist-packages
Requires: numpy, scipy, smart-open
Required-by: 


In [ ]:
!python --version

Python 3.11.11


In [ ]:

#product_relevant_comments_20250401_233547
######THE BESTTTTTTT
import os
import glob
import re
import nltk
import string
import numpy as np
import pandas as pd
import textwrap
from datetime import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
    nltk.data.find('tokenizers/punkt_tab') # Check if punkt_tab is downloaded.
except LookupError:
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('punkt_tab') # Download punkt_tab.

# Global variables for caching
stop_words = set(stopwords.words('english'))
cached_embeddings = {}

headphone_file = "/content/Headphones.xlsx"
sheet_name = "Headphone-Avail-Price"

def safe_string(text):
    """Convert any input to a string safely"""
    if pd.isna(text):
        return ""
    return str(text)

def contains_product_name(text, title, product_name):
    """
    Check if product name is mentioned in either the text or title
    Returns True only if at least part of the product name is found
    """
    text = safe_string(text).lower()
    title = safe_string(title).lower()
    product_name = safe_string(product_name).lower()

    # Check for exact product name
    if product_name in text or product_name in title:
        return True

    # Check for product components
    product_parts = product_name.split()

    required_matches = max(1, len(product_parts) // 1)

    # Count matches in text
    text_matches = sum(1 for part in product_parts
                      if len(part) > 2 and part not in stop_words and part in text)

    # Count matches in title
    title_matches = sum(1 for part in product_parts
                       if len(part) > 2 and part not in stop_words and part in title)

    # Check if we have enough matches in either text or title
    return (text_matches >= required_matches) or (title_matches >= required_matches)

def extract_key_terms(description):
    """Extract key product terms from the description"""
    text = safe_string(description).lower()

    # Remove punctuation and tokenize
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(text)

    # Remove stopwords and keep only alphanumeric words
    key_terms = [word for word in words if word.isalnum()
                and word not in stop_words
                and len(word) > 2]  # Filter out very short words

    # Return unique terms
    return set(key_terms)

def extract_feature_phrases(text):
    """Extract feature phrases from product description"""
    text = safe_string(text)
    lines = text.split('\n')

    features = []
    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Remove any leading identifiers (like bullets, numbers)
        line = re.sub(r'^[•\-\*\d]+\.?\s*', '', line)

        # Check for key feature indicators
        if ':' in line:
            # Split by colon and take the meaningful part
            parts = line.split(':', 1)
            feature = parts[0].strip()
            if feature and len(feature.split()) <= 5:  # Reasonable feature name length
                features.append(feature.lower())

            # Also add the full line
            features.append(line.lower())
        else:
            # Add full line as a feature if not too long
            if len(line.split()) <= 10:
                features.append(line.lower())

    # Add additional features that are part of a list or technical specs
    if len(features) < 3 and len(lines) >= 3:
        # This might be a paragraph description, try to extract noun phrases
        combined_text = ' '.join(lines)
        tokens = word_tokenize(combined_text.lower())
        important_terms = [word for word in tokens if word.isalnum() and word not in stop_words]
        important_bigrams = [f"{tokens[i]} {tokens[i+1]}" for i in range(len(tokens)-1)
                           if tokens[i].isalnum() and tokens[i+1].isalnum()
                           and tokens[i] not in stop_words]

        features.extend(important_terms[:10])  # Add top single terms
        features.extend(important_bigrams[:5])  # Add some bigrams

    return list(set(features))  # Remove duplicates

def compute_feature_overlap(comment, product_features):
    """Compute how many product features are mentioned in the comment"""
    comment_text = safe_string(comment).lower()

    # Direct substring match (exact feature mentioned)
    matches = 0
    for feature in product_features:
        if feature in comment_text:
            matches += 1
            continue

        # Check for key parts of multi-word features (e.g., "noise cancellation" -> "cancellation")
        feature_parts = feature.split()
        if any(part in comment_text for part in feature_parts if len(part) > 3 and part not in stop_words):
            matches += 0.5
            continue

    return matches

def get_embedding(text, model):
    """Get embedding with caching"""
    text = safe_string(text)

    if text in cached_embeddings:
        return cached_embeddings[text]

    embedding = model.encode(text)
    cached_embeddings[text] = embedding
    return embedding

def is_quality_comment(comment, min_words=15, min_sentences=2):
     # Check if a comment meets quality criteria for length and substance
    # Convert to string and handle NaN values
    comment = safe_string(comment)

    if not comment:
        return False

    # Remove URLs
    comment_text = re.sub(r'https?://\S+|www\.\S+', '', comment)

    # Remove markdown formatting
    comment_text = re.sub(r'[*_~`#\[\]\(\)\{\}>]', '', comment_text)

    # Count words (excluding stopwords)
    words = [w for w in word_tokenize(comment_text.lower())
             if w.isalnum() and w not in stop_words]

    # Count sentences
    sentences = sent_tokenize(comment_text)

    # Check quality criteria
    return (len(words) >= min_words and
            len(sentences) >= min_sentences and
            any(len(s.split()) > 5 for s in sentences))  # At least one substantial sentence

def product_focused_similarity(product_description, comment, product_name, product_features, model):
    # Calculate product-focused similarity score
    product_description = safe_string(product_description)
    comment = safe_string(comment)
    product_name = safe_string(product_name)

    # Calculate feature overlap
    feature_overlap = compute_feature_overlap(comment, product_features)

    # Calculate semantic similarity
    prod_embedding = get_embedding(product_description, model)
    comment_embedding = get_embedding(comment, model)
    semantic_sim = cosine_similarity([prod_embedding], [comment_embedding])[0][0]

    # Product mention factor
    product_mention = 1 if product_name.lower() in comment.lower() else 0.5

    # Calculate final score with emphasis on feature overlap
    final_score = (0.4 * semantic_sim +
                  0.4 * min(1.0, feature_overlap / max(1, len(product_features) / 3)) +
                  0.2 * product_mention)

    return {
        'final_score': final_score,
        'semantic_similarity': semantic_sim,
        'feature_overlap': feature_overlap,
        'product_mention': product_mention
    }

def score_and_rank_comments(df, product_description, product_name, model, batch_size=100):
    # Score and rank comments with progress bar and batching
    # First, filter out AutoModerator comments
    if 'comment_author' in df.columns:
        initial_count = len(df)
        df = df[df['comment_author'] != 'AutoModerator'].copy()
        print(f"Removed {initial_count - len(df)} AutoModerator comments")
    else:
        print("Warning: 'comment_author' column not found, can't filter AutoModerator comments")

    print(f"Original comment count: {len(df)}")

    # Check column existence and create safe getters
    def safe_get(row, column, default=""):
        if column in row and not pd.isna(row[column]):
            return row[column]
        return default

    # Apply quality filter
    df['is_quality'] = df.apply(
        lambda row: is_quality_comment(safe_get(row, 'comment_body')),
        axis=1
    )

    filtered_df = df[df['is_quality']].copy()
    print(f"Quality comment count: {len(filtered_df)} (removed {len(df) - len(filtered_df)} low-quality comments)")

    if filtered_df.empty:
        return pd.DataFrame()  # Return empty frame if no good comments

    # Check for product name mention in title or comment
    filtered_df['mentions_product'] = filtered_df.apply(
        lambda row: contains_product_name(
            safe_get(row, 'comment_body'),
            safe_get(row, 'post_title'),
            product_name
        ),
        axis=1
    )

    # Filter to only include comments with product mentions
    product_mention_df = filtered_df[filtered_df['mentions_product']].copy()
    print(f"Comments mentioning product: {len(product_mention_df)} (removed {len(filtered_df) - len(product_mention_df)} without product mention)")

    if product_mention_df.empty:
        return pd.DataFrame()  # Return empty frame if no comments mention the product

    # Extract product features once
    key_terms = extract_key_terms(product_description)
    product_features = extract_feature_phrases(product_description)
    print(f"Extracted {len(key_terms)} key terms and {len(product_features)} features")
    print(f"Sample features: {', '.join(product_features[:3])}")

    total_rows = len(product_mention_df)
    scores = []

    # Process in batches to show progress
    for i in tqdm(range(0, total_rows, batch_size), desc=f"Processing {product_name}"):
        batch = product_mention_df.iloc[i:min(i+batch_size, total_rows)]
        batch_scores = [
            product_focused_similarity(
                product_description,
                safe_get(row, 'comment_body'),
                product_name,
                product_features,
                model
            ) for _, row in batch.iterrows()
        ]
        scores.extend(batch_scores)

    score_df = pd.DataFrame(scores)
    result_df = pd.concat([product_mention_df.reset_index(drop=True), score_df], axis=1)

    # Sort and filter by minimum score
    result_df = result_df.sort_values(by='final_score', ascending=False)
    result_df = result_df[result_df['final_score'] > 0.3]  # Only keep reasonably relevant comments

    # Mark how product was mentioned (in title, body, or both)
    result_df['product_in_title'] = result_df.apply(
        lambda row: product_name.lower() in safe_string(row.get('post_title', '')).lower() or
                   any(part in safe_string(row.get('post_title', '')).lower()
                      for part in product_name.lower().split()
                      if len(part) > 2 and part not in stop_words),
        axis=1
    )

    result_df['product_in_body'] = result_df.apply(
        lambda row: product_name.lower() in safe_string(row.get('comment_body', '')).lower() or
                   any(part in safe_string(row.get('comment_body', '')).lower()
                      for part in product_name.lower().split()
                      if len(part) > 2 and part not in stop_words),
        axis=1
    )

    return result_df

def format_comment(text, max_length=None):
    # Format a comment for display with word wrap
    if max_length and len(text) > max_length:
        return text[:max_length] + "..."
    return text

def print_top_comments(product_name, comments_df, count=5):
    # Print the top comments for a product with nice formatting
    if comments_df.empty:
        print(f"\n{'='*80}")
        print(f"No relevant comments found for {product_name}")
        print(f"{'='*80}")
        return

    print(f"\n{'='*80}")
    print(f"TOP {min(count, len(comments_df))} COMMENTS FOR: {product_name}")
    print(f"{'='*80}")

    top_comments = comments_df.head(count)

    for i, (_, comment) in enumerate(top_comments.iterrows(), 1):
        print(f"\n{i}. Score: {comment['final_score']:.2f} | Features: {comment['feature_overlap']:.1f}")

        # Show where product is mentioned
        mention_locations = []
        if comment.get('product_in_title', False):
            mention_locations.append("title")
        if comment.get('product_in_body', False):
            mention_locations.append("body")
        mention_str = f"Product mentioned in: {', '.join(mention_locations)}"
        print(f"   {mention_str}")

        print(f"   {'-'*76}")

        # Print post title
        post_title = safe_string(comment.get('post_title', ''))
        if post_title:
            print(f"   Title: {post_title[:70]}")
            print()

        # Get comment text
        comment_text = safe_string(comment.get('comment_body', ''))

        # Print wrapped comment text with proper indentation
        lines = textwrap.wrap(comment_text, width=75)
        for line in lines:
            print(f"   {line}")

        print(f"   {'-'*76}")

def main():
    # Load model once
    print("Loading sentence transformer model...")
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Look for both stroller and headphone files
    possible_files = glob.glob("All_Headphone_Reviews_Reddit_Comments_*.xlsx")
    if not possible_files:
        print("No comment files found. Please make sure your data file exists.")
        return

    # Use the most recent file
    latest_file = max(possible_files, key=os.path.getmtime)
    print(f"Loading comments from: {latest_file}")

    # Print columns to help diagnose issues
    try:
        comments_df = pd.read_excel(latest_file)
        print(f"Loaded {len(comments_df)} comments")
        print(f"DataFrame columns: {comments_df.columns.tolist()}")
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return

    # Check for required columns
    required_columns = ['comment_body', 'post_title', 'comment_author']
    missing_columns = [col for col in required_columns if col not in comments_df.columns]
    if missing_columns:
        print(f"Warning: Missing required columns: {missing_columns}")
        print("Available columns:", comments_df.columns.tolist())
        # Try to map similar column names
        column_mapping = {}
        for req_col in missing_columns:
            potential_matches = [col for col in comments_df.columns
                                if req_col.lower() in col.lower() or
                                col.lower() in req_col.lower()]
            if potential_matches:
                column_mapping[req_col] = potential_matches[0]
                print(f"Using '{potential_matches[0]}' for '{req_col}'")

        # Rename columns if possible
        if column_mapping:
            comments_df = comments_df.rename(columns={v: k for k, v in column_mapping.items()})

    # Product definitions
    try:
        spec_df = pd.read_excel("Headphones.xlsx", sheet_name="Headphone-Avail-Price")
        print("Loaded product specifications from 'Headphones.xlsx'.")

        # Ensure necessary columns exist
        if not {'Product Name', 'Specifications'}.issubset(spec_df.columns):
            print("Missing required columns in 'Headphone-Avail-Price' sheet.")
            return

        # Create dictionary of product names to specifications
        products = dict(zip(spec_df['Product Name'].astype(str), spec_df['Specifications'].astype(str)))
        print(f"Loaded {len(products)} product definitions.")

    except Exception as e:
        print(f"Error reading product specifications: {e}")
        return

    # Process each product and print top 5 comments
    ranked_comments = {}
    for pn in products.keys():
        print(f"\nProcessing {pn}...")
        ranked_df = score_and_rank_comments(comments_df, products[pn], pn, model)

        if len(ranked_df) > 0:
            ranked_comments[pn] = ranked_df
            print(f"Found {len(ranked_df)} relevant comments for {pn}")
        else:
            ranked_comments[pn] = pd.DataFrame()  # Empty DataFrame
            print(f"No relevant comments found for {pn}")

        # Print top 5 comments right away
        print_top_comments(pn, ranked_df, count=5)

    summary_data = []
    for pn, df in ranked_comments.items():
        if not df.empty:
            # Drop duplicate comments
            unique_comments = df.drop_duplicates(subset='comment_body')

            # Sort and take top 5
            top_comments = unique_comments.sort_values(by='final_score', ascending=False).head(5)

            # Collect for final output
            top_comments['product_name'] = pn
            summary_data.append(top_comments)

    # Save to Excel
    if summary_data:
        all_ranked_top_comments = pd.concat(summary_data)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"top5_unique_comments_per_product_{timestamp}.xlsx"
        all_ranked_top_comments.to_excel(filename, index=False)
        print(f"\n✅ Saved top 5 unique comments per product to: {filename}")
    else:
        print("\nNo relevant comments found for any product.")

if __name__ == "__main__":
    main()

Loading sentence transformer model...
Loading comments from: All_Headphone_Reviews_Reddit_Comments_20250404_231032.xlsx
Loaded 14500 comments
DataFrame columns: ['product', 'post_title', 'post_id', 'post_url', 'post_score', 'post_created_utc', 'subreddit', 'comment_id', 'comment_author', 'comment_body', 'comment_score', 'comment_created_utc', 'comment_url', 'comment_depth', 'is_top_level']
Loaded product specifications from 'Headphones.xlsx'.
Loaded 145 product definitions.

Processing Sennheiser HD 800 S...
Removed 250 AutoModerator comments
Original comment count: 14250
Quality comment count: 6256 (removed 7994 low-quality comments)
Comments mentioning product: 4 (removed 6252 without product mention)
Extracted 90 key terms and 20 features
Sample features: connectivity technology: wired  is discontinued by manufacturer ‏ : ‎ no, connectivity technology, open back, around ear, reference class dynamic headphone


Processing Sennheiser HD 800 S: 100%|██████████| 1/1 [00:00<00:00,  2.17it/s]


Found 2 relevant comments for Sennheiser HD 800 S

TOP 2 COMMENTS FOR: Sennheiser HD 800 S

1. Score: 0.34 | Features: 2.0
   Product mentioned in: title, body
   ----------------------------------------------------------------------------
   Title: Worth Upgrading my DAC/Amp For Sennheiser HD 800 S?

   I totally agree with a tube amp complementing the HD800S (because I have 1
   and have both solid state and Darkvoice), but I don't think it's the
   upgrade path, more like a sideways jump to another endless time sink of
   tube rolling (ask me how I know).   I've got the Massdrop x Grace Design
   Standard DAC Balanced to Drop THX 789 amp and don't feel I'll never need
   another DAC or Solid State Amp (not sure any of these are available now)
   along with the Darkvoice 336 tube amp and 20 some tubes.   Schiit makes
   great gear, I have their Jotenheim, absolutely great. Powers the HD800s no
   problem. I think it's worth upgrading to another solid state amp, maybe not
   the DAC, 

Processing Sony WH-1000XM4: 100%|██████████| 1/1 [00:04<00:00,  4.02s/it]


Found 38 relevant comments for Sony WH-1000XM4

TOP 5 COMMENTS FOR: Sony WH-1000XM4

1. Score: 0.81 | Features: 7.0
   Product mentioned in: body
   ----------------------------------------------------------------------------
   Title: What are the best Noise-Cancelling Headsets to buy today?

   I’ve tried a few different ones, but the Sony WH-1000XM4 is probably my top
   pick right now. The noise cancellation is top-notch, and they're crazy
   comfortable even after wearing them for hours. Plus, the battery life is
   pretty insane—I think I get about 30 hours on a single charge. They handle
   different noise environments really well, which is awesome if you're moving
   around a lot.  Another solid pick is the Bose QuietComfort 35 II. They’re
   also super comfy and the noise cancelling is pretty close to the Sony’s.
   They might not have all the latest tech features, but they're really
   reliable. If you’re a Bose fan, you can’t really go wrong there.  Some
   folks also recomm

...

Found 8 relevant comments for Audio-Technica ATH-M50xBT2

TOP 5 COMMENTS FOR: Audio-Technica ATH-M50xBT2

1. Score: 0.49 | Features: 7.0
   Product mentioned in: title
   ----------------------------------------------------------------------------
   Title: Advice on Sony WH-1000XM4 or Audio-Technica ATH-M50xBT2

   I wear it for several hours during long flights and train rides, its very
   comfortable as long as I position it correctly, if I throw it on any old
   how one of my ears brushes against the mesh over the driver, after an hour
   it gets quite sore. I don't have wing nut ears, they're pretty average
   size. It occasionally gets a bit warm but I tend to overheat easily, I'm
   built for frigid temperatures.   Its a great product though I wouldn't
   recommend any ANC headphone to someone who only wants the best possible
   sound for the money, that isn't what these products are for, they're
   travel/commuter cans with many very useful features. All of the features
   work

Processing Dali IO-12: 100%|██████████| 1/1 [00:01<00:00,  1.49s/it]


Found 12 relevant comments for Dali IO-12

TOP 5 COMMENTS FOR: Dali IO-12

1. Score: 0.73 | Features: 10.0
   Product mentioned in: title, body
   ----------------------------------------------------------------------------
   Title: B&O Beoplay H100 vs. Dali IO-12

   I compared them and honestly it wasn't really all that close. I do have to
   state some biases of mine. I own Dali tower speakers and IO6s (even though
   my first week with 1st pair required an exchange due to hiss).   I
   auditioned both headphones in Copenhagen. The Dali at a HiFi store and the
   H100 at the B&O Copenhagen location. The HiFi store did sell the H100, but
   didn't have an audition set available. I found the experience at the B&O
   store somewhat unpleasant right off the bat. The guy in a suit who insisted
   on talking my ear off was unneeded. Both were auditioned in a very quiet
   environment for as much time as I wanted. Sadly while the Dalis were able
   to connect to both my phone via Bluetoot

Processing Grado GW100x: 100%|██████████| 1/1 [00:00<00:00,  6.62it/s]


Found 3 relevant comments for Grado GW100x

TOP 3 COMMENTS FOR: Grado GW100x

1. Score: 0.34 | Features: 2.0
   Product mentioned in: title, body
   ----------------------------------------------------------------------------
   Title: Disappointed in Grado SR325x

   They might not be the right headphone for you. But if you’re interested in
   hanging onto them and experimenting, I would recommend investing in several
   pairs of aftermarket pads. I have about eight different pad sets, most of
   which cost me $10-$15, and each pad turns my Grados into a different
   headphone. I have the SR60, SR80, SR125X, SR325X, and GW100X. The 125X and
   325X are my favorites, and I never use the stock pads on any of them. Hope
   you find what you’re looking for!
   ----------------------------------------------------------------------------

2. Score: 0.34 | Features: 2.0
   Product mentioned in: title, body
   ----------------------------------------------------------------------------
   Tit

Loading sentence transformer model...
Loading comments from: All_Stroller_Reviews_Reddit_Comments_20250402_214315.xlsx
Loaded 500 comments
DataFrame columns: ['product', 'post_title', 'post_id', 'post_url', 'post_score', 'post_created_utc', 'subreddit', 'comment_id', 'comment_author', 'comment_body', 'comment_score', 'comment_created_utc', 'comment_url', 'comment_depth', 'is_top_level']

Processing Sennheiser HD 800 S...
Removed 6 AutoModerator comments
Original comment count: 494
Quality comment count: 259 (removed 235 low-quality comments)
Comments mentioning product: 4 (removed 255 without product mention)
Extracted 44 key terms and 16 features
Sample features: open back, around ear, reference class dynamic headphone, dynamic, headphone


Processing Sennheiser HD 800 S: 100%|██████████| 1/1 [00:00<00:00,  2.22it/s]


Found 2 relevant comments for Sennheiser HD 800 S

TOP 2 COMMENTS FOR: Sennheiser HD 800 S

1. Score: 0.43 | Features: 2.5
   Product mentioned in: title, body
   ----------------------------------------------------------------------------
   Title: Worth Upgrading my DAC/Amp For Sennheiser HD 800 S?

   Haha, honestly really good thoughts. Seems like you have a better view than
   I on these solid states, I defer to your opinion on that. My thought on
   tubes is that there's a lot more testing and research to find your desired
   sound from tubes, which may not be to OPs interest. You can get the HD800
   to really sing with tubes, but the wrong tubes and you won't have as good
   an experience. That is all. Good feedback.
   ----------------------------------------------------------------------------

2. Score: 0.34 | Features: 2.0
   Product mentioned in: title, body
   ----------------------------------------------------------------------------
   Title: Worth Upgrading my DAC/Amp

Processing Sony WH-1000XM4: 100%|██████████| 1/1 [00:00<00:00,  1.12it/s]


Found 5 relevant comments for Sony WH-1000XM4

TOP 5 COMMENTS FOR: Sony WH-1000XM4

1. Score: 0.51 | Features: 1.0
   Product mentioned in: title, body
   ----------------------------------------------------------------------------
   Title: Sony WH-1000XM4 vs Bose QC Ultra (Review)

   Hi, I can choose between getting the Bose QuietComfort Ultra or AirPods Pro
   2 for free.  I already own the Sony WH-1000XM4. Are the differences in
   noise cancellation between the Bose and the XM4 significant enough to go
   with the Bose, or should I stick with the XM4 and choose the AirPods Pro 2
   instead?  Thanks in advance!
   ----------------------------------------------------------------------------

2. Score: 0.50 | Features: 2.0
   Product mentioned in: title
   ----------------------------------------------------------------------------
   Title: Sony WH-1000XM4 vs Bose QC Ultra (Review)

   I had all three (including the XM5). My brief opinions: The Bose were the
   most comfortable and

...

No relevant comments found for Beyerdynamic Amiron Wireless

No relevant comments found for Beyerdynamic Amiron Wireless

Processing Focal Stellia...
Removed 6 AutoModerator comments
Original comment count: 494
Quality comment count: 259 (removed 235 low-quality comments)
Comments mentioning product: 39 (removed 220 without product mention)
Extracted 21 key terms and 10 features
Sample features: impedance: 35 ω, loudspeakers, frequency response


Processing Focal Stellia: 100%|██████████| 1/1 [00:03<00:00,  3.42s/it]


Found 10 relevant comments for Focal Stellia

TOP 5 COMMENTS FOR: Focal Stellia

1. Score: 0.39 | Features: 1.0
   Product mentioned in: title, body
   ----------------------------------------------------------------------------
   Title: Auditioned the Focal Clear MG (damn) and Focal Stellia (damn) today. M

   The comparison was in reference to dynamics and not headphone types.   I
   suggested you the headphones for your dynamics requirement as per my
   personal opinions. The ones I have heard and own.  I forgot one, the
   Aurorus Borealis. One of the best dynamics I have heard and own, with a
   great stock tuning.  And yes the Stellia are also great. Liked them but for
   me they too are over priced for their performance.  Reviews and subjective
   experience are two very different things. Whatever headphones you get, make
   sure to give them a patient listen. For reference here, don't know what
   reviews you read mentioning Arya V3 as dull especially compared to Clear
   MG, 

Processing Audeze LCD-X: 100%|██████████| 1/1 [00:05<00:00,  5.77s/it]

Found 41 relevant comments for Audeze LCD-X

TOP 5 COMMENTS FOR: Audeze LCD-X

1. Score: 0.66 | Features: 3.0
   Product mentioned in: title, body
   ----------------------------------------------------------------------------
   Title: Just auditioned Audeze LCD-X and saw the Fostex th900mk2 signature edi

   About the lcd x consider the weight. If you use the headphones for a long
   time you will notice it, it bothered me.    As already mentioned you will
   need to use equalization to get a more correct frequency, this will always
   limit you to use equipment that has an available and comfortable
   equalization or the Audeze plugin.   You also have to consider that
   although they have a low impedance, they are not so easy to move. With some
   good amplification a planar always improves.   The rewards when well in
   conjunction with gear and good EQ: they are less dark than others in the
   line, have fantastic separation, the timbre is not bad and the soundstage
   is not wid